<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/evaluation_metrics_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SentencePiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from utils import *
import json
# from datasets import Dataset, load_dataset

ModuleNotFoundError: ignored

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

## Dataset processing

In [ ]:
with open("./dev.json") as input_file:
      valid_data = json.load(input_file)

def ad_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"context": table_text, "question": question, "answer": steps_text, "id":example["id"]}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

valid_pd = pd.DataFrame(ad_data(valid_data,False))

Net Errors: 35


In [ ]:
step_pattern = r"Step \d+"
import re
# def num_steps_dict_fn(example):
#     example["num_steps"] = len(re.findall(step_pattern, example["answer"]))
#     return example

In [ ]:
# data_splits = {'dev': valid_data}
valid_pd = pd.DataFrame(ad_data(valid_data,False))
valid_pd["num_steps"] = valid_pd["answer"].apply(lambda x: len(re.findall(step_pattern, x)))
# dataset_hf = {"valid": Dataset.from_pandas(valid_pd)}

Net Errors: 35


In [ ]:
# dataset_hf['valid']=dataset_hf['valid'].map(num_steps_dict_fn)

In [ ]:
# dataset_hf_single = dataset_hf['valid'].filter(lambda example: example['num_steps']==1)

In [ ]:
# from torch.utils.data import Dataset as Dataset_torch

In [66]:
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("./finqa_finetune_t5.pth/",from_tf=False)
BATCH_SIZE = 8
TESTING = False
MODEL.to(DEVICE);

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OSError: ignored

In [ ]:
from utils import FinQA_Dataset

In [ ]:
MAX_Q_LEN = 1024
MAX_A_LEN = 128

finqa_valid1 = FinQA_Dataset(TOKENIZER, valid_pd, MAX_Q_LEN, MAX_A_LEN)
finqa_valid1

In [ ]:
len(finqa_valid1)

# Inference

## Inefernce on 1 step validation data

In [ ]:
# from torch.utils.data import DataLoader as DataLoader_torch

In [ ]:
test_loader = DataLoader(finqa_valid1, batch_size=4)

In [ ]:
MODEL.eval();
label_texts = []
generated_texts = []
for batch in tqdm(test_loader, desc="Generation batches"):
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
    with torch.no_grad():
        # Use the `generate` method for text generation
        generated_output = MODEL.generate(
            input_ids=input_ids,
            max_length=128,  # Set the desired maximum length for generated text
            num_beams=5,    # You can adjust the number of beams for beam search
            temperature=0.8  # You can adjust the temperature for sampling,
            )
        for i,output_i in enumerate(generated_output):
          generated_text = TOKENIZER.decode([x for x in output_i if x!= -100], skip_special_tokens=True)
          generated_texts.append(generated_text)
          label_text = batch["labels"][i]
          label_texts.append(label_text)

## Postprocessing